In [1]:
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=59b2bb7a0855f7793522cd07803affac675345bbae115ae947f2fc148b71b5ec
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [2]:
from google.colab import drive
drive.mount('/content/Drive')
%cd Drive/MyDrive/
#!git clone https://github.com/Skar0/fire-detection
!cd /content/Drive/MyDrive/fire-detection
API_key = "" # @param {type:"string"}

Mounted at /content/Drive
/content/Drive/MyDrive


In [3]:
import collections.abc
import numpy as np
from keras.models import load_model
import keras.utils as image
from keras.applications.inception_v3 import preprocess_input as inception_preprocess_input

collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

classes = ['fire', 'no_fire', 'start_fire']

async def predict(image_path, model_path="/content/Drive/MyDrive/fire-detection/transfer_learned_model.h5"):
  model = load_model(model_path)
  img = image.load_img(image_path, target_size=(224, 224, 3))
  # processed image to feed the network
  processed_img = image.img_to_array(img)
  processed_img = np.expand_dims(processed_img, axis=0)
  processed_img = inception_preprocess_input(processed_img)
  # get prediction using the network
  predictions = model.predict(processed_img)[0]
  return {clas:"{:.8f}".format(float(x)) for (clas,x) in zip(classes,predictions)}

In [26]:
from fastapi import FastAPI, File, UploadFile
from typing import Annotated
import requests
from starlette.middleware.cors import CORSMiddleware as CORSMiddleware
import nest_asyncio
from pyngrok import ngrok
import uvicorn

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

async def getImage(lon,lat,zoom):
  url = "https://dev.virtualearth.net/REST/V1/Imagery/Map/AerialWithLabels/{}%2C{}/{}?mapSize=550%2C500&format=jpeg&key={}".format(lat,lon,zoom,API_key)
  response = requests.get(url)
  with open("/content/image.jpg", "wb") as f:
    f.write(response.content)


@app.get('/{lon}/{lat}/{zoom}')
async def query(lon,lat,zoom):
  await getImage(lon,lat,zoom)
  answer = await predict("/content/image.jpg")
  return {'answer':answer}

@app.post("/")
async def create_file(data: UploadFile=File()):
    return {
        "file_size": len(data),
    }

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)
ngrok.disconnect(ngrok_tunnel.public_url)

Public URL: https://3d9e-34-68-41-177.ngrok.io


INFO:     Started server process [912]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


1/1 [==============================] - 2s 2s/step
INFO:     2401:4900:61bb:f607:6ac4:aaa3:6af5:644d:0 - "GET /76.71286964110912/14.140681988910245/14 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [912]


In [ ]:
await predict("/content/fire.jpg","/content/Drive/MyDrive/fire-detection/transfer_learned_model.h5")

1/1 [==============================] - 2s 2s/step


{'fire': '0.99999940', 'no_fire': '0.00000065', 'start_fire': '0.00000004'}